<a href="https://colab.research.google.com/github/DefaultaideN/EncodeBoosting/blob/main/encode_boosting.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!pip install seaborn
#!pip install tensorflow
#!pip install tf_keras_vis

In [ ]:
#Load some packages
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras

print("Tensorflow Edition:", tf.__version__)
seed = 2020
np.random.seed(seed)
tf.random.set_seed(seed)
print("Packages Loaded!")

Tensorflow Edition: 2.4.1
Packages Loaded!


In [ ]:
#inspect GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
n_GPUs = len(physical_devices)
print("Num_GPUs_Available:", n_GPUs)
if n_GPUs>0:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)

Num_GPUs_Available: 1


In [ ]:
!nvidia-smi

Tue Feb  2 13:02:22 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8     9W /  70W |     10MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#load data from Google Drive
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import joblib
class train_encode_booster():
  
  def __init__(self, data_path, save_path):
    self.save_path = save_path
    self.data_path = data_path
    self.encoded_list = []
    self.labels_list = []
    self.pca_model_list = []
    self.pca_encode_list = []
    self.data_pca_fullencode = []

  def loading(self):
    for model_no in range(4):
      file_name = SAVE_PATH+'model_'+str(model_no)+'/pca_transformer.joblib'
      pca_model = joblib.load(file_name)
      self.pca_model_list.append(pca_model)

      for data_part in range(3):
        encoded_name = self.save_path+'model_'+str(model_no)+'/part'+str(data_part)+'_encode.csv'
        single_encode = np.genfromtxt(encoded_name, delimiter=',')
        pca_encode = pca_model.transform(single_encode[:, :-1])

        self.encoded_list.append(single_encode[:, :-1])
        self.pca_encode_list.append(pca_encode)
        if model_no == 3:
          self.labels_list.append(single_encode[:, -1])

    for data_part in range(3):
      whole_pca = self.pca_encode_list[0*3+data_part]
      for model_no in range(1, 4):
        whole_pca = np.append(whole_pca, self.pca_encode_list[model_no*3+data_part], axis=1)
      self.data_pca_fullencode.append(whole_pca)

    return
  





In [ ]:
SAVE_PATH = '/content/drive/MyDrive/paper_file/model_trained/'
DATA_PATH = '/content/drive/MyDrive/paper_file/data/'
NAME_LIST = ['VGG', 'Xcept', 'Efcb5', 'Res50']

In [ ]:
booster = train_encode_booster(DATA_PATH, SAVE_PATH)
booster.loading()

In [ ]:
import tensorflow as tf
from tensorflow import keras
import keras.layers as layers

In [ ]:
def convert_label(label, num):
  n = label.shape[0]
  y_train = np.zeros((n, num))
  for i in range(n):
    cate = int(label[i])
    y_train[i, cate] = 1
  
  return y_train
  
X_train = booster.data_pca_fullencode[0]
y_train = convert_label(booster.labels_list[0], 4)
X_val = booster.data_pca_fullencode[1]
y_val = convert_label(booster.labels_list[1], 4)
X_test = booster.data_pca_fullencode[2]
y_test = convert_label(booster.labels_list[2], 4)

In [ ]:
n_fullencode = booster.data_pca_fullencode[0].shape[1]
final_model = keras.Sequential([
    keras.Input(shape=(n_fullencode, )),
    layers.BatchNormalization(),                            
    layers.Dense(units=1024, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(units=512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(units=256, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),
    layers.Dense(units=4, activation='softmax', kernel_regularizer='l1_l2'),          
])

In [ ]:
final_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
final_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 498)               1992      
_________________________________________________________________
dense (Dense)                (None, 1024)              510976    
_________________________________________________________________
batch_normalization_1 (Batch (None, 1024)              4096      
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
batch_normalization_2 (Batch (None, 512)               2048      
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0

In [ ]:
final_model.fit(x=X_train, y=y_train, epochs=30, validation_data=(X_val, y_val), verbose=1)

Epoch 1/30
346/346 [==============================] - 5s 6ms/step - loss: 1.2485 - accuracy: 0.8171 - val_loss: 0.3435 - val_accuracy: 0.9712
Epoch 2/30
346/346 [==============================] - 2s 5ms/step - loss: 0.2813 - accuracy: 0.9808 - val_loss: 0.2173 - val_accuracy: 0.9721
Epoch 3/30
346/346 [==============================] - 2s 5ms/step - loss: 0.1811 - accuracy: 0.9906 - val_loss: 0.2026 - val_accuracy: 0.9718
Epoch 4/30
346/346 [==============================] - 2s 5ms/step - loss: 0.1554 - accuracy: 0.9902 - val_loss: 0.1977 - val_accuracy: 0.9728
Epoch 5/30
346/346 [==============================] - 2s 5ms/step - loss: 0.1438 - accuracy: 0.9919 - val_loss: 0.1920 - val_accuracy: 0.9705
Epoch 6/30
346/346 [==============================] - 2s 5ms/step - loss: 0.1296 - accuracy: 0.9939 - val_loss: 0.1830 - val_accuracy: 0.9737
Epoch 7/30
346/346 [==============================] - 2s 5ms/step - loss: 0.1152 - accuracy: 0.9961 - val_loss: 0.1906 - val_accuracy: 0.9699
Epoch 

In [ ]:
final_model.evaluate(x=X_test, y=y_test)

50/50 [==============================] - 0s 2ms/step - loss: 0.1953 - accuracy: 0.9639


[0.19528260827064514, 0.963878333568573]

In [ ]:
result = final_model.predict(X_test)

In [ ]:
from sklearn.metrics import roc_auc_score
score = roc_auc_score(y_test, result)

In [ ]:
score, result.shape

(0.9936816839210496, (1578, 4))

In [ ]:
result

array([[9.9832577e-01, 8.2237739e-04, 4.2720060e-04, 4.2461365e-04],
       [9.4577456e-01, 4.2931687e-02, 4.8685437e-03, 6.4252922e-03],
       [9.9900085e-01, 4.9004477e-04, 2.6648914e-04, 2.4260730e-04],
       ...,
       [3.3147198e-03, 3.4716157e-03, 1.9236831e-02, 9.7397679e-01],
       [1.4886478e-04, 2.5169435e-04, 1.0180684e-04, 9.9949765e-01],
       [5.7841360e-04, 4.0801187e-04, 2.1081365e-04, 9.9880266e-01]],
      dtype=float32)

In [ ]:
from numpy import argmax
from numpy import mean
from sklearn.metrics import confusion_matrix
label_index = []
result_index = []
for label in y_test:
  label_index.append(argmax(label))
for result_ in result:
  result_index.append(argmax(result_))

cnf_matrix = confusion_matrix(label_index, result_index)
#print(cnf_matrix)
#[[1 1 3]
# [3 2 2]
# [1 3 1]]

FP = cnf_matrix.sum(axis=0) - np.diag(cnf_matrix)  
FN = cnf_matrix.sum(axis=1) - np.diag(cnf_matrix)
TP = np.diag(cnf_matrix)
TN = cnf_matrix.sum() - (FP + FN + TP)

FP = FP.astype(float)
FN = FN.astype(float)
TP = TP.astype(float)
TN = TN.astype(float)

# Sensitivity, hit rate, recall, or true positive rate
TPR = TP/(TP+FN)
# Specificity or true negative rate
TNR = TN/(TN+FP) 
# Precision or positive predictive value
PPV = TP/(TP+FP)
# Negative predictive value
NPV = TN/(TN+FN)
# Fall out or false positive rate
FPR = FP/(FP+TN)
# False negative rate
FNR = FN/(TP+FN)
# False discovery rate
FDR = FP/(TP+FP)
# Overall accuracy
ACC = (TP+TN)/(TP+FP+FN+TN)
print('Model: '+'DNN'+' | '+'Precision: '+str(PPV)+' | '+'Specificity: '+str(TNR))

Model: DNN | Precision: [0.94153846 0.94827586 0.99618321 0.95171026] | Specificity: [0.98492063 0.9910045  0.99810606 0.97785978]


In [ ]:
'''
#load model
model_loaded = []
for i in range(4):
  model = keras.models.load_model(SAVE_PATH+'model_'+str(i))
  model_loaded.append(model)
'''

"\n#load model\nmodel_loaded = []\nfor i in range(4):\n  model = keras.models.load_model(SAVE_PATH+'model_'+str(i))\n  model_loaded.append(model)\n"

In [ ]:
'''
#get pca model
import joblib
from sklearn.decomposition import PCA


def find_n(explained, fixed_percentage):
  sum = 0
  i = 0
  while sum<=fixed_percentage:
    sum += explained[i]
    i += 1
  return i


for model_no in range(4):
  for data_part in range(3):
    encoded = booster.encoded_list[model_no*3+data_part]
    labels = booster.labels_list[model_no*3+data_part]

    decomposer_pca = PCA(n_components=500)
    de_pca = decomposer_pca.fit_transform(encoded)

    n_de = find_n(decomposer_pca.explained_variance_ratio_, 0.95)
    decomposer_pca = PCA(n_components=n_de)
    de_pca = decomposer_pca.fit_transform(encoded)

    file_name = SAVE_PATH+'model_'+str(model_no)+'/pca_transformer.joblib'
    joblib.dump(decomposer_pca, file_name)
'''

"\n#get pca model\nimport joblib\nfrom sklearn.decomposition import PCA\n\n\ndef find_n(explained, fixed_percentage):\n  sum = 0\n  i = 0\n  while sum<=fixed_percentage:\n    sum += explained[i]\n    i += 1\n  return i\n\n\nfor model_no in range(4):\n  for data_part in range(3):\n    encoded = booster.encoded_list[model_no*3+data_part]\n    labels = booster.labels_list[model_no*3+data_part]\n\n    decomposer_pca = PCA(n_components=500)\n    de_pca = decomposer_pca.fit_transform(encoded)\n\n    n_de = find_n(decomposer_pca.explained_variance_ratio_, 0.95)\n    decomposer_pca = PCA(n_components=n_de)\n    de_pca = decomposer_pca.fit_transform(encoded)\n\n    file_name = SAVE_PATH+'model_'+str(model_no)+'/pca_transformer.joblib'\n    joblib.dump(decomposer_pca, file_name)\n"

In [ ]:
'''
def plot_encode(encode, labels, part):
  model_name = 'Part '+str(part)+': model_full_encode'
  fig = plt.figure(part, figsize=[3*7, 7])
  axes = fig.subplots(1, 3)

  from sklearn.manifold import TSNE
  decomposer_tsne = TSNE(n_components=2)
  de_tsne = decomposer_tsne.fit_transform(encode)
  scatter = axes[0].scatter(x=de_tsne[:,0], y=de_tsne[:,1], c=labels)
  legend1 = axes[0].legend(*scatter.legend_elements(), loc="best", title="Classes")
  axes[0].set_title(model_name+'---Tsne')
  axes[0].add_artist(legend1)

  from sklearn.manifold import Isomap
  decomposer_iso = Isomap(n_components=2, n_neighbors=10)
  de_iso = decomposer_iso.fit_transform(encode)
  scatter = axes[1].scatter(x=de_iso[:,0], y=de_iso[:,1], c=labels)
  legend1 = axes[1].legend(*scatter.legend_elements(), loc="best", title="Classes")
  axes[1].set_title(model_name+'---Isomap')
  axes[1].add_artist(legend1)

  from sklearn.decomposition import PCA
  decomposer_pca = PCA(n_components=2)
  de_pca = decomposer_pca.fit_transform(encode)
  scatter = axes[2].scatter(x=de_pca[:,0], y=de_pca[:,1], c=labels)
  legend1 = axes[2].legend(*scatter.legend_elements(), loc="best", title="Classes")
  axes[2].set_title(model_name+'---PCA')
  axes[2].add_artist(legend1)
  print('Done!')

plot_encode(booster.data_pca_fullencode[0], booster.labels_list[0], 0)
'''

'\ndef plot_encode(encode, labels, part):\n  model_name = \'Part \'+str(part)+\': model_full_encode\'\n  fig = plt.figure(part, figsize=[3*7, 7])\n  axes = fig.subplots(1, 3)\n\n  from sklearn.manifold import TSNE\n  decomposer_tsne = TSNE(n_components=2)\n  de_tsne = decomposer_tsne.fit_transform(encode)\n  scatter = axes[0].scatter(x=de_tsne[:,0], y=de_tsne[:,1], c=labels)\n  legend1 = axes[0].legend(*scatter.legend_elements(), loc="best", title="Classes")\n  axes[0].set_title(model_name+\'---Tsne\')\n  axes[0].add_artist(legend1)\n\n  from sklearn.manifold import Isomap\n  decomposer_iso = Isomap(n_components=2, n_neighbors=10)\n  de_iso = decomposer_iso.fit_transform(encode)\n  scatter = axes[1].scatter(x=de_iso[:,0], y=de_iso[:,1], c=labels)\n  legend1 = axes[1].legend(*scatter.legend_elements(), loc="best", title="Classes")\n  axes[1].set_title(model_name+\'---Isomap\')\n  axes[1].add_artist(legend1)\n\n  from sklearn.decomposition import PCA\n  decomposer_pca = PCA(n_components=

In [ ]:
'''
#check labels
for data_part in range(3):
  check01 = booster.labels_list[0*3+data_part] == booster.labels_list[1*3+data_part]
  check02 = booster.labels_list[0*3+data_part] == booster.labels_list[2*3+data_part]
  for i in range(check01.shape[0]):
    if check01[i]:
      pass
    else:
      print(i)

  for i in range(check01.shape[0]):
    if check02[i]:
      pass
    else:
      print(i)
'''

'\n#check labels\nfor data_part in range(3):\n  check01 = booster.labels_list[0*3+data_part] == booster.labels_list[1*3+data_part]\n  check02 = booster.labels_list[0*3+data_part] == booster.labels_list[2*3+data_part]\n  for i in range(check01.shape[0]):\n    if check01[i]:\n      pass\n    else:\n      print(i)\n\n  for i in range(check01.shape[0]):\n    if check02[i]:\n      pass\n    else:\n      print(i)\n'

In [ ]:
while True:pass

KeyboardInterrupt: ignored